# 01_CMS_CostReport_EDA
## Overview
This notebook explores and prepares CMS Hospital Provider Cost Report data for use in analyzing
the relationship between hospital staffing capacity and quality outcomes.

The primary objective is to construct hospital-level staffing intensity measures that can be
linked to CMS quality ratings and regional nursing workforce data in later stages of the project.

## Data Source & Scope
**Source:** CMS Hospital Provider Cost Report  
**Unit of analysis:** Hospital fiscal-year observation  
**Key identifier:** Provider CCN

This dataset contains detailed financial, utilization, and staffing information reported annually
by hospitals to CMS. Fiscal years vary by hospital and do not necessarily align with calendar years.


## 1) Load and Clean 
This section loads the raw CMS cost report data and performs basic cleaning steps required for
consistent identification and numeric analysis.

In [6]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", 200)

cost = pd.read_csv("CMS_HospitalProviderCostReport.csv", low_memory=False)

# Ensure CCN formatting
cost["Provider CCN"] = cost["Provider CCN"].astype(str).str.zfill(6)

# Convert numeric columns
num_cols = [
    "FTE - Employees on Payroll",
    "Total Days (V + XVIII + XIX + Unknown)",
    "Number of Beds"
]

for c in num_cols:
    cost[c] = pd.to_numeric(cost[c], errors="coerce")

## 2) Compute staffing metrics

### Interpretation of Staffing Metrics
Two staffing intensity measures are constructed:

- **FTE per 1,000 patient days**: captures staffing relative to inpatient utilization
- **FTE per bed**: captures staffing relative to hospital capacity

These measures use total employees on payroll (not nursing-specific), serving as a proxy for
overall staffing intensity. Extreme values are expected for very small hospitals or facilities
with limited inpatient volume.


In [7]:
cost["fte_per_1000_days"] = np.where(
    cost["Total Days (V + XVIII + XIX + Unknown)"] > 0,
    (cost["FTE - Employees on Payroll"] /
     cost["Total Days (V + XVIII + XIX + Unknown)"]) * 1000,
    np.nan
)

cost["fte_per_bed"] = np.where(
    cost["Number of Beds"] > 0,
    cost["FTE - Employees on Payroll"] / cost["Number of Beds"],
    np.nan
)

cost[["fte_per_1000_days", "fte_per_bed"]].describe(percentiles=[.01, .05, .5, .95, .99])


,fte_per_1000_days,fte_per_bed
count,5935.000000,5941.000000
mean,93.895991,10.444594
std,833.752042,120.908313
min,0.109481,0.004500
1%,4.917820,0.817261
5%,7.049028,1.573333
50%,30.475509,5.182000
95%,164.329907,15.322778
99%,556.662430,26.900541
max,44720.000000,6997.200000


## 3. Missingness, Data Quality, and Duplicates

### Notes on Missingness
High missingness is expected for financial and utilization subcomponents that are not applicable
to all hospital types. Staffing and bed counts exhibit substantially lower missingness, supporting
their use in downstream analysis.

In [8]:
cost.isna().mean().sort_values(ascending=False).head(15)

Outlier Payments For Discharges                        1.000000
DRG Amounts Other Than Outlier Payments                1.000000
Hospital Total Days Title V For Adults & Peds          0.980993
Total Discharges Title V                               0.980010
Hospital Total Discharges Title V For Adults & Peds    0.980010
Total Days Title V                                     0.979191
Notes Receivable                                       0.960347
Unsecured Loans                                        0.948714
Wage-Related Costs (RHC/FQHC)                          0.937408
Wage Related Costs for Part - A Teaching Physicians    0.931509
Health Information Technology Designated Assets        0.927740
Mortgage Payable                                       0.888088
Wage Related Costs for Interns and Residents           0.859905
Minor Equipment Depreciable                            0.846141
Net Revenue from Stand-Alone CHIP                      0.844011
dtype: float64

In [9]:
cost["Provider CCN"].duplicated().mean()

np.float64(0.01032279206947403)

## 4. Final Dataset for Merging

### Output Design
The cleaned dataset retains only identifiers, staffing inputs, and derived staffing metrics
required for merging with CMS quality ratings and BLS workforce data. This minimizes file size
and reduces downstream complexity.


In [10]:
cost_clean = cost[
    [
        "Provider CCN",
        "Hospital Name",
        "State Code",
        "Medicare CBSA Number",
        "Fiscal Year Begin Date",
        "Fiscal Year End Date",
        "FTE - Employees on Payroll",
        "Total Days (V + XVIII + XIX + Unknown)",
        "Number of Beds",
        "fte_per_1000_days",
        "fte_per_bed",
    ]
].copy()

cost_clean.to_csv("cost_report_clean.csv", index=False)